In [6]:
import cv2
import numpy as np

In [8]:
frame = np.ones((480, 480, 3))
frame = cv2.resize(frame, (500, 500))
frame = np.uint8(frame)
print(frame.shape)
gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
print(gray.shape)

(500, 500, 3)
(500, 500)


In [7]:
import cv2

# Open a connection to the camera (0 is the default camera)
cap = cv2.VideoCapture(0)

# Check if the camera opened successfully
if not cap.isOpened():
    print("Error: Could not open camera.")
    exit()

# Loop to continuously get frames
while True:
    # Capture frame-by-frame
    frame = np.ones((480, 480, 3), dtype=np.uint8)

    # Display the resulting frame
    cv2.imshow('Camera Test', frame)

    # Break the loop on 'q' key press
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the camera and close all OpenCV windows
cap.release()
cv2.destroyAllWindows()


array([[[121, 160, 188],
        [126, 166, 192],
        [123, 163, 188],
        ...,
        [123, 153, 195],
        [115, 144, 187],
        [120, 149, 192]],

       [[121, 160, 188],
        [125, 165, 191],
        [118, 158, 184],
        ...,
        [118, 147, 189],
        [119, 148, 190],
        [126, 155, 197]],

       [[122, 161, 189],
        [116, 155, 183],
        [121, 160, 188],
        ...,
        [119, 148, 190],
        [119, 145, 188],
        [121, 148, 190]],

       ...,

       [[ 75,  79,  94],
        [ 70,  76,  91],
        [ 58,  66,  81],
        ...,
        [ 72,  87,  94],
        [ 73,  88,  95],
        [ 79,  94, 101]],

       [[ 70,  74,  89],
        [ 64,  69,  84],
        [ 56,  64,  79],
        ...,
        [ 80,  95, 102],
        [ 77,  91,  99],
        [ 78,  93, 100]],

       [[ 66,  69,  84],
        [ 67,  73,  88],
        [ 53,  61,  76],
        ...,
        [ 85, 100, 107],
        [ 79,  94, 101],
        [ 83,  97, 105]]